In [1]:
import pyproj
from  pyproj  import  CRS
from pyproj import Proj
import h5py
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, colors
import glob
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import pandas as pd
import gzip
from collections import defaultdict

In [2]:
# 设置空间分辨率
resolution = 10000

# 将WGS 84坐标（4326）转化为等距圆柱投影（4088）
crs = CRS.from_epsg(4326)
crs = CRS.from_string("epsg:4326")
crs = CRS.from_proj4("+proj=latlon")
crs = CRS.from_user_input(4326)
crs2 = CRS.from_epsg(4088)
crs2 = CRS.from_string("epsg:4088")
crs2 = CRS.from_proj4("+proj=latlon")
crs2 = CRS.from_user_input(4088)

transformer = pyproj.Transformer.from_crs(crs,crs2)
transformer_back = pyproj.Transformer.from_crs(crs2,crs)

In [119]:
# 获取数组的长和宽
def get_nlat_nlon_npaeqd(resolution):
    nlat, nlon =40000000/ resolution,40000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon
    
    
def get_nlat_nlon_cyl(resolution):
    nlat, nlon =40000000/ resolution,20000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon

# 获取用来存放数据的grid数组，及其对应的grid_num数组（用来存放一个格子中有多少重合点），以及对应的初始time数组
def get_swh_grid(nlat, nlon):
    swh_grid = np.full(shape=(nlon,nlat), fill_value=np.nan)
    swh_grid_num = np.zeros((nlon,nlat))
    swh_grid_time = np.full(shape=(nlon,nlat), fill_value=np.nan)
    return swh_grid, swh_grid_num, swh_grid_time

def get_num_grid(nlat, nlon):
    num_grid = np.zeros((nlon,nlat))
    return num_grid

# 填充每个点的数值，将有重复数据的点标记到swh_grid_num中,最后求平均
def coincident_point_mean(value_array, grid_array, grid_num_array, projlats, projlons):
    for i in range(len(value_array)):
        x = int((projlons[i] )/resolution)
        y = int(projlats[i]/resolution)
        if grid_num_array[x][y] == 0:
            grid_array[x][y] = value_array[i]
            grid_num_array[x][y] += 1
        else:
            grid_array[x][y] += value_array[i]
            grid_num_array[x][y] += 1
    grid_array = grid_array / grid_num_array
    return grid_array

# def coincident_time_log(value_array, grid_array, grid_num_array, grid_time_array, projlats, projlons):
#     time_dict = {}
#     for i in range(len(value_array)):
#         x = int((projlons[i] )/resolution)
#         y = int(projlats[i]/resolution)
#         if grid_num_array[x][y] == 0:
#             hy_grid[x][y] = hy_value_array_masked[i]
#             hy_time_grid[x][y] = hy_time_array[i]
#             dict_name = str(x) + '+' + str(y)
#             hy_time_dict[dict_name] = {hy_time_grid[x][y]: hy_grid[x][y]}
#             grid_num_array[x][y] = 1
        
#         else:
#             dict_name = str(x) + '+' + str(y)
#             hy_time_dict[dict_name][hy_time_array[i]] = hy_value_array_masked[i]
#             grid_num_array[x][y] += 1
        
#     # 删除没有重合点的子字典
#     for key in list(hy_time_dict.keys()):
#         if len(hy_time_dict[key].items()) < 2 :
#             del hy_time_dict[key]

In [4]:
# hy_dir_path = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711'
hy_dir_path_01 = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711\20200711'
hyfiles = glob.glob(hy_dir_path_01 + '\*.nc')
hy_lon_array = np.array([])
hy_lat_array = np.array([])
hy_time_array = np.array([])
hy_value_array = np.array([])
hy_swhc_mask_array = np.array([])

for hyfile in hyfiles:
    with Dataset(hyfile, mode='r') as fh:
        lons = fh.variables['lon'][:]
        lats = fh.variables['lat'][:]
        swhc = fh.variables['swh_c'][:]
        time = fh.variables['time'][:]
        swhc_mask = swhc.mask
        hy_lon_array = np.append(hy_lon_array, lons)
        hy_lat_array = np.append(hy_lat_array, lats)
        hy_value_array = np.append(hy_value_array, swhc)
        hy_time_array = np.append(hy_time_array, time)
        hy_swhc_mask_array = np.append(hy_swhc_mask_array, swhc_mask)
    hy_value_array_masked = np.ma.array(hy_value_array, mask=hy_swhc_mask_array)
hy_df = pd.DataFrame([hy_lon_array, hy_lat_array, hy_time_array, hy_value_array_masked], index=['lon', 'lat', 'time', 'value'])

In [5]:
hy_projlats,hy_projlons = transformer.transform(hy_lat_array,hy_lon_array)
# 获取东西半球的nlat，nlon，is_grid_west,is_grid_num_west
hy_nlat, hy_nlon =get_nlat_nlon_cyl(resolution)
hy_grid, hy_num_grip, hy_time_grid = get_swh_grid(hy_nlat*2, hy_nlon*2)

In [6]:
hy_time_dict = {}
for i in range(len(hy_value_array_masked)):
    x = int((hy_projlons[i] )/resolution)
    y = int(hy_projlats[i]/resolution)
    if hy_num_grip[x][y] == 0:
        hy_grid[x][y] = hy_value_array_masked[i]
        hy_time_grid[x][y] = hy_time_array[i]
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name] = {hy_time_grid[x][y]: hy_grid[x][y]}
        hy_num_grip[x][y] = 1
        
    else:
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name][hy_time_array[i]] = hy_value_array_masked[i]
        hy_num_grip[x][y] += 1
        
'''# 删除没有重合点的子字典
for key in list(hy_time_dict.keys()):
    if len(hy_time_dict[key].items()) < 2 :
        del hy_time_dict[key]'''

<ipython-input-6-c951ff59db16>:6: UserWarning: Warning: converting a masked element to nan.
  hy_grid[x][y] = hy_value_array_masked[i]


'# 删除没有重合点的子字典\nfor key in list(hy_time_dict.keys()):\n    if len(hy_time_dict[key].items()) < 2 :\n        del hy_time_dict[key]'

In [7]:
hy_mean_grid = coincident_point_mean(hy_value_array_masked, hy_grid, hy_num_grip, hy_projlats,hy_projlons)

<ipython-input-3-54fb976da36e>:31: UserWarning: Warning: converting a masked element to nan.
  grid_array[x][y] += value_array[i]


# Cryosat 数据处理

In [8]:
# cy_dir_path = r'G:\remote_sensing_data\CRYOSAT-2\20200701_20200711'
cy_dir_path_01 = r"G:\remote_sensing_data\CRYOSAT-2\2020\07\20200711"
cyfiles = glob.glob(cy_dir_path_01 + '\*.nc')
cy_lon_array = np.array([])
cy_lat_array = np.array([])
cy_swh_array = np.array([])
cy_swh_mask_array = np.array([])
cy_time_array = np.array([])

for cyfile in cyfiles:
    with Dataset(cyfile, mode='r') as fh:
        lons = fh.variables['lon_01'][:]
        lats = fh.variables['lat_01'][:]
        swh = fh.variables['swh_ocean_01_ku'][:]
        time = fh.variables['time_cor_01'][:]
        swh_mask = swh.mask
        if np.array(swh_mask,dtype = bool).shape == ():
            swh_mask = np.full((len(swh), ), False)
        cy_lon_array = np.append(cy_lon_array, lons)
        cy_lat_array = np.append(cy_lat_array, lats)
        cy_swh_array = np.append(cy_swh_array, swh)
        cy_swh_mask_array = np.append(cy_swh_mask_array, swh_mask)
        cy_time_array = np.append(cy_time_array, time)
cy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)


In [9]:
'''cy_lower_index = []
for i in range(len(cy_swh_array_masked)):
    if cy_lat_array[i] > 60 or cy_lat_array[i] < -60:
        cy_lower_index.append(i)

cy_lat_array = np.delete(cy_lat_array, cy_lower_index)
cy_lon_array = np.delete(cy_lon_array, cy_lower_index)
cy_swh_array = np.delete(cy_swh_array, cy_lower_index)
cy_swh_mask_array = np.delete(cy_swh_mask_array, cy_lower_index)
cy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)'''

'cy_lower_index = []\nfor i in range(len(cy_swh_array_masked)):\n    if cy_lat_array[i] > 60 or cy_lat_array[i] < -60:\n        cy_lower_index.append(i)\n\ncy_lat_array = np.delete(cy_lat_array, cy_lower_index)\ncy_lon_array = np.delete(cy_lon_array, cy_lower_index)\ncy_swh_array = np.delete(cy_swh_array, cy_lower_index)\ncy_swh_mask_array = np.delete(cy_swh_mask_array, cy_lower_index)\ncy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)'

In [9]:
cy_projlats,cy_projlons = transformer.transform(cy_lat_array,cy_lon_array)

# 获取东西半球的nlat，nlon，cy_grid_west,cy_grid_num_west
cy_nlat, cy_nlon =get_nlat_nlon_cyl(resolution)
cy_grid, cy_num_grip, cy_time_grid = get_swh_grid(cy_nlat*2, cy_nlon*2)


In [10]:
cy_time_dict = {}
for i in range(len(cy_swh_array_masked)):
    x = int((cy_projlons[i] )/resolution)
    y = int(cy_projlats[i]/resolution)
    if cy_num_grip[x][y] == 0:
        cy_grid[x][y] = cy_swh_array_masked[i]
        cy_time_grid[x][y] = cy_time_array[i]
        dict_name = str(x) + '+' + str(y)
        cy_time_dict[dict_name] = {cy_time_grid[x][y]: cy_grid[x][y]}
        cy_num_grip[x][y] = 1
        
    else:
        dict_name = str(x) + '+' + str(y)
        cy_time_dict[dict_name][cy_time_array[i]] = cy_swh_array_masked[i]
        cy_num_grip[x][y] += 1
'''
# 删除没有重合点的子字典
for key in list(cy_time_dict.keys()):
    if len(cy_time_dict[key].items()) < 2 :
        del cy_time_dict[key]'''

<ipython-input-10-893a831ee7fa>:6: UserWarning: Warning: converting a masked element to nan.
  cy_grid[x][y] = cy_swh_array_masked[i]


'\n# 删除没有重合点的子字典\nfor key in list(cy_time_dict.keys()):\n    if len(cy_time_dict[key].items()) < 2 :\n        del cy_time_dict[key]'

In [120]:
'''cy_mean_num_grid = get_num_grid(cy_nlat*2, cy_nlon*2)
cy_mean_grid = coincident_point_mean(cy_swh_array_masked, cy_grid, cy_mean_num_grid, cy_projlats,cy_projlons)'''

<ipython-input-119-3b7e03c2948e>:32: UserWarning: Warning: converting a masked element to nan.
  grid_array[x][y] = value_array[i]
<ipython-input-119-3b7e03c2948e>:35: UserWarning: Warning: converting a masked element to nan.
  grid_array[x][y] += value_array[i]


IndexError: index 60429 is out of bounds for axis 0 with size 60429

# 处理icesat数据

In [40]:
# icesat2 的时间是从2018-01-01 00：00：00开始记的，Hy2b和cryosat2的是从2000-01-01 00：00：00开始计的
# 2018-01-01 00：00：00 与2000-01-01 00：00：00 相差了568080000秒
correction_second = 568080000

dir_path_01 = r'G:\remote_sensing_data\ICESAT-2\2020\07\20200711'
ncfiles = glob.glob(dir_path_01 + '\*.h5')
is_lon_array = np.array([])
is_lat_array = np.array([])
is_value_array = np.array([])
is_time_array = np.array([])
tracks = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']
for ncfile in ncfiles:
    with h5py.File(ncfile, 'r') as f:
        for track in tracks:
            lats = f[track]['ssh_segments']['latitude'][:]
            lons = f[track]['ssh_segments']['longitude'][:]
            value = f[track]['ssh_segments']['heights']['swh'][:]
            time = f[track]['ssh_segments']['delta_time'][:]
            
            is_lon_array = np.append(is_lon_array, lons)
            is_lat_array = np.append(is_lat_array, lats)
            is_value_array = np.append(is_value_array, value)
            is_time_array = np.append(is_time_array, time + correction_second)

In [113]:
is_projlats,is_projlons = transformer.transform(is_lat_array,is_lon_array)

# 获取东西半球的nlat，nlon，is_grid_west,is_grid_num_west
is_nlat, is_nlon =get_nlat_nlon_cyl(resolution)
is_grid, is_num_grip, is_time_grid = get_swh_grid(is_nlat*2, is_nlon*2)


In [117]:
is_time_dict = {}
n=1
for i in range(len(is_value_array)):
    x = int((is_projlons[i] )/resolution)
    y = int(is_projlats[i]/resolution)
    if is_num_grip[x][y] == 0:
        is_grid[x][y] = is_value_array[i]
        is_time_grid[x][y] = is_time_array[i]
        dict_name = str(x) + '+' + str(y)
        is_time_dict[dict_name] = {is_time_grid[x][y]: is_grid[x][y]}
        is_num_grip[x][y] = 1
            
    else:
        dict_name = str(x) + '+' + str(y)
        is_time_dict[dict_name][is_time_array[i]] = is_value_array[i]
        is_num_grip[x][y] += 1


In [ ]:
'''is_mean_grid = coincident_point_mean(is_value_array, is_grid, is_num_grip, is_projlats,is_projlons)
'''

# 对比hy与cryosat交叉点


In [13]:
# 将两个grid数组相减，还有数据的部分为交叉点
hy_cy_grid = hy_mean_grid - cy_mean_grid
hy_cy_diff_grid = np.argwhere(np.isnan(hy_cy_grid)!= True)

# 设置时间窗口为3600秒
time_window = 3600

# 创建一个用来存储点对的list
hy_cy_coincident_list = []

for cy_time_key in cy_time_dict['-607+-1611'].keys():
    print(cy_time_key)
    print(cy_time_dict['-607+-1611'][cy_time_key])

cy_time_dict['-607+-1611']

In [27]:
for i in range(len(hy_cy_diff_grid)): 
    diff_x = hy_cy_diff_grid[i][0]
    if diff_x > 2000: diff_x -= 4000
    diff_y = hy_cy_diff_grid[i][1]
    if diff_y > 4000: diff_y -= 8000
    key = str(diff_x) + '+' + str(diff_y)
    
    for cy_time_key in cy_time_dict[key].keys():
        for hy_time_key in hy_time_dict[key].keys():
            if np.abs(hy_time_key - cy_time_key)< time_window :
                coincident_value = []
                coincident_value.append(key)
                coincident_value.append(hy_time_key)
                coincident_value.append(hy_time_dict[key][hy_time_key])
                coincident_value.append(cy_time_key)
                coincident_value.append(cy_time_dict[key][cy_time_key])
                hy_cy_coincident_list.append(coincident_value)


22138.89441907406
22419.376847982407
22418.376937031746
22415.490156054497
22416.43359398842
22412.603391885757
22411.603497862816
22413.54682803154
22412.5469340086
22409.6601729393
22408.660290956497
21947.713413000107
21950.650217056274
21949.65156507492
21951.593654990196
21950.595003008842
21954.53438591957
21953.535740971565
21955.47781395912
21954.479169011116
21957.419893980026
21959.361969947815
21753.568584918976
21752.57018494606
21754.5120139122
21753.513613939285
21756.4538500309
21759.394078969955
21758.395683050156
21760.33751785755
21759.33912193775
21762.279349923134
21763.222785949707
21765.16461598873
63440.05760204792
63442.00033593178
63442.99963200092
63445.88580393791
63444.94236803055
63447.828540086746
63448.82783806324
22990.55730497837
22989.55823993683
22991.500733971596
22990.501668930054
22986.672298908234
22987.61573600769
22984.729791998863
22983.73072206974
22980.842278003693
22981.785717964172
63252.34947991371
63251.406042933464
63254.2924079895
63256

In [28]:
# 输出符合条件的点
len(coincident_list)

0

In [26]:
'''hy_overlap_swh = np.array([])
cy_overlap_swh = np.array([])
diff_value = np.array([])
for i in range(len(hy_cy_diff_grid)):
    hy_overlap_swh = np.append(hy_overlap_swh, hy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]])
    cy_overlap_swh = np.append(cy_overlap_swh, cy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]])
    diff_value = np.append(diff_value, round(hy_cy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]], 8))
'''

In [27]:
'''# 将每个交叉点的数值记入DataFrame

hy_cy_data={'HY-2B swh':hy_overlap_swh,
      'CRYOSAT swh':cy_overlap_swh,
      'Diff':diff_value
     }
hy_cy_df = pd.DataFrame(hy_cy_data)'''

# 对比hy与icesat交叉点

In [122]:
# 将两个grid数组相减，还有数据的部分为交叉点
hy_is_grid = hy_mean_grid - is_mean_grid
hy_is_diff_grid = np.argwhere(np.isnan(hy_is_grid)!= True)


# 创建一个用来存储点对的list
hy_is_coincident_list = []


for cy_time_key in cy_time_dict['-607+-1611'].keys():
    print(cy_time_key)
    print(cy_time_dict['-607+-1611'][cy_time_key])

cy_time_dict['-607+-1611']

In [133]:
for i in range(len(hy_is_diff_grid)): 
    diff_x = hy_is_diff_grid[i][0]
    if diff_x > 2000: diff_x -= 4000
    diff_y = hy_is_diff_grid[i][1]
    if diff_y > 4000: diff_y -= 8000
    key = str(diff_x) + '+' + str(diff_y)
    
    for is_time_key in is_time_dict[key].keys():
        for hy_time_key in hy_time_dict[key].keys():
            if np.abs(hy_time_key - is_time_key)< time_window :
                coincident_value = []
                coincident_value.append(key)
                coincident_value.append(hy_time_key)
                coincident_value.append(hy_time_dict[key][hy_time_key])
                coincident_value.append(is_time_key)
                coincident_value.append(is_time_dict[key][is_time_key])
                hy_is_coincident_list.append(coincident_value)


In [135]:
# 输出符合条件的点
len(hy_is_coincident_list)

126